# 파이선 활용 2 : Pandas와 Folium을 활용한 지도 시각화

## 2.1 : 전국무료급식소표준데이터 위도, 경도 활용 시각화

### 관련 라이브러리 import

In [1]:
import folium
import glob
import os
import json
import pandas as pd
import numpy as np
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

### csv 전체 데이터 import

In [2]:
path = 'data/'
allFiles = glob.glob(os.path.join(path+"*.csv"))

df = pd.DataFrame()
for file in allFiles:

    print(file)

# csv 파일은 인코딩 문제로 한글 텍스트의 깨짐이 있을 수 있습니다.
# 아래에서 pickle 형식으로 데이터 형식을 변환해 문제를 해결했습니다.

    temp = pd.read_csv(file, engine='python', encoding = 'utf-8')
    df = pd.concat([df, temp])

data\전국무료급식소표준데이터_원본_02.csv


### import 결과 확인

In [3]:
df.head()

,시설명,소재지도로명주소,소재지지번주소,운영기관명,전화번호,급식장소,급식대상,급식시간,급식요일,운영시작일자,운영종료일자,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,수서종합사회복지관,서울특별시 강남구 광평로 56길 11,서울특별시 강남구 수서동 723,수서종합사회복지관,02-451-7244,수서종합사회복지관,만60세이상 결식우려가 있는 강남구 거주 저소득 어르신,중식(11:30~12:30),NaN,NaN,NaN,37.487979,127.105631,2020-03-01,3220000,서울특별시 강남구
1,대청종합사회복지관,서울특별시 강남구 양재대로 55길 10,서울특별시 강남구 일원동 711,대청종합사회복지관,02-459-6332,대청종합사회복지관,만60세이상 결식우려가 있는 강남구 거주 저소득 어르신,중식(11:00~12:00),NaN,NaN,NaN,37.493156,127.090993,2020-03-01,3220000,서울특별시 강남구
2,세곡리엔파크4단지 경로당,서울특별시 강남구 헌릉로590길 88,서울특별시 강남구 세곡동 522,사회복지법인 영산,02-565-3857,세곡리엔파크4단지 경로당,만60세이상 결식우려가 있는 강남구 거주 저소득 어르신,중식(11:30~12:30),NaN,NaN,NaN,37.462877,127.101445,2020-03-01,3220000,서울특별시 강남구
3,대청경로당,서울특별시 강남구 개포로124길 24,서울특별시 강남구 일원동 621,사회복지법인 영산,02-565-3857,대청경로당,만60세이상 결식우려가 있는 강남구 거주 저소득 어르신,중식(11:30~12:30),NaN,NaN,NaN,37.493198,127.083000,2020-03-01,3220000,서울특별시 강남구
4,대치1단지(일원2) 경로당,서울특별시 강남구 개포로 109길 5,서울특별시 강남구 개포동 12,사회복지법인 영산,02-565-3857,대치1단지(일원2) 경로당,만60세이상 결식우려가 있는 강남구 거주 저소득 어르신,중식(11:30~12:30),NaN,NaN,NaN,37.492331,127.074591,2020-03-01,3220000,서울특별시 강남구


### 컬럼 항목 수로 정상 import 여부 재확인

In [4]:
print('컬럼 항목 수 :', len(df.columns))
list(df.columns)

컬럼 항목 수 : 16


['시설명',
 '소재지도로명주소',
 '소재지지번주소',
 '운영기관명',
 '전화번호',
 '급식장소',
 '급식대상',
 '급식시간',
 '급식요일',
 '운영시작일자',
 '운영종료일자',
 '위도',
 '경도',
 '데이터기준일자',
 '제공기관코드',
 '제공기관명']

### 필요한 컬럼만 불러와 데이터셋 간략화

In [5]:
dataset = df[['시설명',
 '소재지도로명주소',
 '위도',
 '경도',             
 '운영기관명']]

#간략화된 데이터셋 확인
dataset.head()

,시설명,소재지도로명주소,위도,경도,운영기관명
0,수서종합사회복지관,서울특별시 강남구 광평로 56길 11,37.487979,127.105631,수서종합사회복지관
1,대청종합사회복지관,서울특별시 강남구 양재대로 55길 10,37.493156,127.090993,대청종합사회복지관
2,세곡리엔파크4단지 경로당,서울특별시 강남구 헌릉로590길 88,37.462877,127.101445,사회복지법인 영산
3,대청경로당,서울특별시 강남구 개포로124길 24,37.493198,127.083000,사회복지법인 영산
4,대치1단지(일원2) 경로당,서울특별시 강남구 개포로 109길 5,37.492331,127.074591,사회복지법인 영산


### pickle 형식으로 csv에서 저장 형식 전환 (인코딩 오류 발생 예방)

In [6]:
dataset.to_pickle("df01.pkl")

### 저장한 pickle 파일 import해 데이터셋 확인

In [7]:
df_pic = pd.read_pickle("df01.pkl")

In [8]:
df_pic

,시설명,소재지도로명주소,위도,경도,운영기관명
0,수서종합사회복지관,서울특별시 강남구 광평로 56길 11,37.487979,127.105631,수서종합사회복지관
1,대청종합사회복지관,서울특별시 강남구 양재대로 55길 10,37.493156,127.090993,대청종합사회복지관
2,세곡리엔파크4단지 경로당,서울특별시 강남구 헌릉로590길 88,37.462877,127.101445,사회복지법인 영산
3,대청경로당,서울특별시 강남구 개포로124길 24,37.493198,127.083000,사회복지법인 영산
4,대치1단지(일원2) 경로당,서울특별시 강남구 개포로 109길 5,37.492331,127.074591,사회복지법인 영산
...,...,...,...,...,...
1437,늘푸른노인대학,경상남도 사천시 용현면 용치로 12,35.011986,128.063070,용현교회
1438,개웅교회,서울특별시 구로구 개봉로20길 91-10,37.493112,126.860017,개웅교회
1439,중국동포의집,서울특별시 구로구 남부순환로 1307,37.478797,126.896075,(사)지구촌사랑나눔
1440,한국지체장애인협회 구로구지회,서울특별시 구로구 디지털로19길 24,37.480799,126.889897,한국지체장애인협회 구로지회


### 기존 CSV 데이터에서 위, 경도 Null 값인 행 삭제

In [9]:
#dataset.drop([154,232,479,985,1199], inplace=True)

In [10]:
dataset

,시설명,소재지도로명주소,위도,경도,운영기관명
0,수서종합사회복지관,서울특별시 강남구 광평로 56길 11,37.487979,127.105631,수서종합사회복지관
1,대청종합사회복지관,서울특별시 강남구 양재대로 55길 10,37.493156,127.090993,대청종합사회복지관
2,세곡리엔파크4단지 경로당,서울특별시 강남구 헌릉로590길 88,37.462877,127.101445,사회복지법인 영산
3,대청경로당,서울특별시 강남구 개포로124길 24,37.493198,127.083000,사회복지법인 영산
4,대치1단지(일원2) 경로당,서울특별시 강남구 개포로 109길 5,37.492331,127.074591,사회복지법인 영산
...,...,...,...,...,...
1437,늘푸른노인대학,경상남도 사천시 용현면 용치로 12,35.011986,128.063070,용현교회
1438,개웅교회,서울특별시 구로구 개봉로20길 91-10,37.493112,126.860017,개웅교회
1439,중국동포의집,서울특별시 구로구 남부순환로 1307,37.478797,126.896075,(사)지구촌사랑나눔
1440,한국지체장애인협회 구로구지회,서울특별시 구로구 디지털로19길 24,37.480799,126.889897,한국지체장애인협회 구로지회


In [11]:
# dataset.to_pickle("df01_clear.pkl")

In [12]:
df_pic = pd.read_pickle("df01_clear.pkl")

In [13]:
df_pic

,시설명,소재지도로명주소,위도,경도,운영기관명
0,수서종합사회복지관,서울특별시 강남구 광평로 56길 11,37.487979,127.105631,수서종합사회복지관
1,대청종합사회복지관,서울특별시 강남구 양재대로 55길 10,37.493156,127.090993,대청종합사회복지관
2,세곡리엔파크4단지 경로당,서울특별시 강남구 헌릉로590길 88,37.462877,127.101445,사회복지법인 영산
3,대청경로당,서울특별시 강남구 개포로124길 24,37.493198,127.083000,사회복지법인 영산
4,대치1단지(일원2) 경로당,서울특별시 강남구 개포로 109길 5,37.492331,127.074591,사회복지법인 영산
...,...,...,...,...,...
1437,늘푸른노인대학,경상남도 사천시 용현면 용치로 12,35.011986,128.063070,용현교회
1438,개웅교회,서울특별시 구로구 개봉로20길 91-10,37.493112,126.860017,개웅교회
1439,중국동포의집,서울특별시 구로구 남부순환로 1307,37.478797,126.896075,(사)지구촌사랑나눔
1440,한국지체장애인협회 구로구지회,서울특별시 구로구 디지털로19길 24,37.480799,126.889897,한국지체장애인협회 구로지회


In [14]:
#json 파일 가져와 행정구역 구분
geo_path = 'data/TL_SCCO_SIG.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

### 전국 무료급식소 Pop-up 시각화

In [16]:
# 시작 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data_size = len(df_pic)

map_freefood = folium.Map(location=loc, zoom_start=12)


#팝업 마커 삽입 (위도, 경도)
for i in range(data_size):

    folium.Marker(list(df_pic.iloc[i][['위도', '경도']]),
                 popup=df_pic.iloc[i][['시설명']],
                 icon=folium.Icon(color='green')).add_to(map_freefood)
    
#html 형식으로 저장해 1.파싱과 웹 연동    
map_freefood.save('markedmap.html')
                 
map_freefood